In [5]:
import cv2
import datetime

# RTSP 주소를 사용하여 VideoCapture 객체 생성
rtsp_url = "rtsp://210.99.70.120:1935/live/cctv001.stream"
cap = cv2.VideoCapture(rtsp_url)

# 동영상 코덱 및 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 설정
out = None  # 동영상 저장 객체

is_recording = False  # 녹화 여부
screenshot_count = 0  # 스크린샷 저장 카운터
zoom_level = 1.0  # 줌 레벨 초기화
pan_x, pan_y = 0, 0  # 팬 좌표 초기화

while True:
    ret, frame = cap.read()  # 카메라에서 프레임 읽기
    
    if not ret:
        print("카메라에서 영상을 읽을 수 없습니다.")
        break

    # 줌 기능 적용 (프레임 크기 조정)
    h, w = frame.shape[:2]
    center_x, center_y = w // 2, h // 2
    new_w, new_h = int(w / zoom_level), int(h / zoom_level)
    cropped_frame = frame[pan_y:pan_y+new_h, pan_x:pan_x+new_w]
    frame = cv2.resize(cropped_frame, (w, h))

    # 타임스탬프 추가 (위치 하단, 색상 검은색으로 변경)
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    font_scale = 1.5  # 날짜 크기를 크게 조정
    cv2.putText(frame, timestamp, (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), 2)  # 검은색

    # 녹화 중일 경우, 빨간색 원을 표시
    if is_recording:
        cv2.circle(frame, (50, 50), 20, (0, 0, 255), -1)
        out.write(frame)  # 프레임을 파일에 저장
    
    # 카메라 영상을 화면에 표시
    cv2.imshow('RTSP Camera', frame)

    # 키 입력 대기 (1ms)
    key = cv2.waitKey(1) & 0xFF

    # ESC 키를 누르면 종료
    if key == 27:
        break
    # Space 키를 누르면 녹화 모드 전환
    elif key == 32:
        if not is_recording:
            is_recording = True
            out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))
            print("녹화 시작")
        else:
            is_recording = False
            out.release()
            print("녹화 종료")
    # 'q' 키로 스크린샷 저장
    elif key == ord('q'):
        screenshot_name = f"screenshot_{screenshot_count}.png"
        cv2.imwrite(screenshot_name, frame)
        print(f"스크린샷 저장: {screenshot_name}")
        screenshot_count += 1
    # '+' 키를 누르면 확대 (줌 인)
    elif key == ord('+'):
        zoom_level = min(zoom_level + 0.1, 3.0)  # 줌 인, 최대 3배까지
    # '-' 키를 누르면 축소 (줌 아웃)
    elif key == ord('-'):
        zoom_level = max(zoom_level - 0.1, 1.0)  # 줌 아웃, 최소 원본 크기
    # 화살표 키로 팬 기능 (좌우 및 상하 이동)
    elif key == ord('w'):  # 위로 이동
        pan_y = max(pan_y - 10, 0)
    elif key == ord('s'):  # 아래로 이동
        pan_y = min(pan_y + 10, h - new_h)
    elif key == ord('a'):  # 왼쪽으로 이동
        pan_x = max(pan_x - 10, 0)
    elif key == ord('d'):  # 오른쪽으로 이동
        pan_x = min(pan_x + 10, w - new_w)

# 자원 해제
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()


스크린샷 저장: screenshot_0.png
